# FGVC Aircraft — CNN feature extraction + SVM (faza 3)
U ovoj fazi koristimo naučeni CNN kao ekstraktor obeležja (feature-a), a zatim treniramo SVM klasifikator
nad tim obeležjima. Uporedimo rezultate sa CNN softmax baseline-om.


In [1]:
# === DIJAGNOSTIKA + ROBUSTAN ROOT/SRC/PROC SETUP ===
import sys
from pathlib import Path

def find_project_root(start: Path) -> Path:
    """
    Penje se naviše dok ne nađe folder koji sadrži 'src' i 'data' (ili dođe do diska).
    Radi čak i ako je notebook pokrenut van 'notebooks/'.
    """
    for p in [start, *start.parents]:
        if (p / "src").exists() and (p / "data").exists():
            return p
    # fallback: ako postoji samo 'src'
    for p in [start, *start.parents]:
        if (p / "src").exists():
            return p
    return start  # poslednja nada

CWD = Path.cwd().resolve()
ROOT = find_project_root(CWD)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("CWD:", CWD)
print("ROOT:", ROOT)
print("src postoji:", (ROOT / "src").exists())
print("data postoji:", (ROOT / "data").exists())

PROC = ROOT / "data" / "processed"
print("PROC:", PROC)
print("train.csv postoji:", (PROC / "train.csv").exists())
print("val.csv   postoji:", (PROC / "val.csv").exists())
print("test.csv  postoji:", (PROC / "test.csv").exists())

CWD: C:\Users\Emilija\Desktop\AircraftClassifier_CNN_SVM\notebooks
ROOT: C:\Users\Emilija\Desktop\AircraftClassifier_CNN_SVM
src postoji: True
data postoji: True
PROC: C:\Users\Emilija\Desktop\AircraftClassifier_CNN_SVM\data\processed
train.csv postoji: True
val.csv   postoji: True
test.csv  postoji: True


In [2]:
# === LAKI IMPORTI (bez TF), PROVERA CSV ===
import pandas as pd
from src.preprocessing import make_datagens, flow_from_dataframe  # test da 'src' radi

# koristi PROC iz prve ćelije
train_df = pd.read_csv(PROC / "train.csv")
val_df   = pd.read_csv(PROC / "val.csv")
test_df  = pd.read_csv(PROC / "test.csv")

print("Train/Val/Test veličine:", len(train_df), len(val_df), len(test_df))
print("Kolone:", train_df.columns.tolist())


Train/Val/Test veličine: 700 150 150
Kolone: ['image', 'label', 'filepath']


In [3]:
# === TEŽI IMPORT (TensorFlow) ===
import tensorflow as tf
from tensorflow.keras.models import load_model

print("TF:", tf.__version__)


TF: 2.20.0


In [5]:
IMG_SIZE = (128, 128)   # koristi istu veličinu kao u CNN treningu
BATCH = 32

# Bez augmentacije, i bez shuffle za sva tri
_, eval_gen = make_datagens(img_size=IMG_SIZE)

train_flow = flow_from_dataframe(eval_gen, train_df, img_size=IMG_SIZE, batch_size=BATCH, shuffle=False)
val_flow   = flow_from_dataframe(eval_gen,  val_df,   img_size=IMG_SIZE, batch_size=BATCH, shuffle=False)
test_flow  = flow_from_dataframe(eval_gen,  test_df,  img_size=IMG_SIZE, batch_size=BATCH, shuffle=False)

print("Broj klasa:", len(train_flow.class_indices))


Found 700 validated image filenames belonging to 10 classes.
Found 150 validated image filenames belonging to 10 classes.
Found 150 validated image filenames belonging to 10 classes.
Broj klasa: 10


## 1) Učitavanje CNN modela i kreiranje feature-extractora
Učitavamo `cnn_baseline.h5` iz faze 2 i kao izlaz uzimamo pretposlednji Dense(128) sloj,
koji koristimo kao vektor obeležja (feature) za SVM.


In [ ]:
# === Load model + robustno pravljenje extractora za Keras 3 ===
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, Sequential

MODEL_PATH = Path("../models/cnn_baseline.h5")   # ako preimenuješ u .keras, promeni ovde
cnn = load_model(MODEL_PATH)

# 1) Ako model nema definisan graph (inputs), "pozovi" ga na dummy tensora da se izgradi
if not getattr(cnn, "inputs", None):
    dummy = tf.zeros((1,) + IMG_SIZE + (3,), dtype=tf.float32)
    _ = cnn(dummy, training=False)  # sada cnn.inputs i cnn.layers[*].output postoje

# 2) Pokušaj standardni extractor (pretposlednji sloj)
try:
    extractor = Model(inputs=cnn.input, outputs=cnn.layers[-2].output)
except Exception as e:
    # Fallback: odseci poslednji Dense sloj (pretpostavka: zadnji je klasifikator)
    feature_layers = cnn.layers[:-1]
    extractor = Sequential(feature_layers)
    # "build" da bi imao definisan izlazni shape
    extractor.build((None,) + IMG_SIZE + (3,))
    print("Extractor napravljen preko Sequential fallback-a:", extractor.output_shape)

extractor.summary()


AttributeError: The layer sequential has never been called and thus has no defined input.